In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("C:/Users/clayton/OneDrive - Blackstone Products LLC/Documents/Python Scripts/School/IS_6950/Items Shipped.csv")
df.columns

C:\Users\clayton\AppData\Local\Temp\ipykernel_14604\680700529.py:1: DtypeWarning: Columns (10,17,21,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Users/clayton/OneDrive - Blackstone Products LLC/Documents/Python Scripts/School/IS_6950/Items Shipped.csv")


Index(['Customer Name', 'Parent', 'Account Manager', 'Item',
       'Item Description', 'Product Group', 'Parent Product Group',
       'Netsuite Link', 'Sales Order', 'Customer PO', 'Tracking Numbers',
       'Invoice Status', 'Date Created', 'Date', 'Ship Date',
       'Import/Domestic', 'Location', 'Is Individual', 'Quantity', 'Amount',
       'Sell Price', 'Shipping State/Province', 'Online Flag', 'Internal ID',
       'Shipping Address'],
      dtype='object')

### Setting Up Data

In [3]:
# converting ship date to pandas datetime
df['Ship Date'] = pd.to_datetime(df['Ship Date'])

# filtering to the conditioner item
item = '5017'
filtered_df = df[(df['Item'] == item) & (df['Import/Domestic'] == 'Domestic')]
# # grouping quantity to frequency of month
new_df = filtered_df.groupby(pd.Grouper(key='Ship Date', freq="M"))['Quantity'].sum().reset_index()
new_df.set_index('Ship Date', inplace=True)
new_df.head()

,Quantity
Ship Date,
2020-01-05,1082.0
2020-01-12,6415.0
2020-01-19,7570.0
2020-01-26,1222.0
2020-02-02,304.0


In [4]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 229 entries, 2020-01-05 to 2024-05-19
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Quantity  229 non-null    float64
dtypes: float64(1)
memory usage: 3.6 KB


In [5]:
from pycaret.time_series import *

exp = TSForecastingExperiment()
exp.setup(data = new_df, target='Quantity', fh=3, coverage=0.90)

,Description,Value
0,session_id,5117
1,Target,Quantity
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(229, 1)"
5,Transformed data shape,"(229, 1)"
6,Transformed train set shape,"(226, 1)"
7,Transformed test set shape,"(3, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [6]:
exp.plot_model(plot="ts")

In [7]:
exp.plot_model(plot="train_test_split")

In [8]:
exp.plot_model(plot="cv")

In [9]:
best_model = exp.compare_models(sort='RMSE') #sort RMSE

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,0.5051,0.3784,6142.6804,7284.8804,0.2993,0.3276,-1.0180,0.0567
naive,Naive Forecaster,0.5178,0.3958,6304.5556,7628.8028,0.2889,0.3692,-1.2351,1.3167
en_cds_dt,Elastic Net w/ Cond. Deseasonalize & Detrending,0.5314,0.3973,6465.0302,7650.3826,0.3371,0.3520,-1.1627,0.1300
ridge_cds_dt,Ridge w/ Cond. Deseasonalize & Detrending,0.5314,0.3973,6465.0302,7650.3826,0.3371,0.3520,-1.1627,0.1333
lasso_cds_dt,Lasso w/ Cond. Deseasonalize & Detrending,0.5314,0.3973,6465.0302,7650.3826,0.3371,0.3520,-1.1627,0.1333
llar_cds_dt,Lasso Least Angular Regressor w/ Cond. Deseasonalize & Detrending,0.5314,0.3973,6465.0302,7650.3826,0.3371,0.3520,-1.1627,0.1333
lr_cds_dt,Linear w/ Cond. Deseasonalize & Detrending,0.5314,0.3973,6465.0302,7650.3826,0.3371,0.3520,-1.1627,0.1300
br_cds_dt,Bayesian Ridge w/ Cond. Deseasonalize & Detrending,0.5389,0.4039,6556.7481,7778.5959,0.3473,0.3580,-1.2265,0.1133
omp_cds_dt,Orthogonal Matching Pursuit w/ Cond. Deseasonalize & Detrending,0.5432,0.4046,6608.7937,7792.3053,0.3504,0.3622,-1.2373,0.0500
stlf,STLF,0.5584,0.4107,6821.7442,7934.6259,0.3884,0.5785,-3.3604,0.0367


In [10]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True})

In [11]:
exp.plot_model(plot="diff", 
               data_kwargs={"lags_list": [[1, 12]], "acf": True, "pacf": True})

In [12]:
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'})

In [13]:
exp.plot_model(plot = 'diagnostics')

## ETS Model

In [14]:
ets_dt = exp.create_model('ets')

exp.plot_model(estimator = ets_dt, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 3})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2024-02-19/2024-02-25,0.2319,0.1696,2860.0692,3297.3381,0.2818,0.2768,0.1449
1,2024-03-11/2024-03-17,0.6074,0.4013,7414.3363,7751.5324,0.4223,0.5405,-7.6546
2,2024-04-01/2024-04-07,1.2018,0.8664,14555.7056,16630.0004,0.4653,0.6405,-2.6963
Mean,NaT,0.6804,0.4791,8276.7037,9226.2903,0.3898,0.4859,-3.4020
SD,NaT,0.3993,0.2898,4813.5043,5542.0306,0.0783,0.1534,3.2230


In [15]:
ets_dt

AutoETS(seasonal='mul', sp=2, trend='add')

In [16]:
exp.check_stats(ets_dt, test = 'all')

,Test,Test Name,Data,Property,Setting,Value
0,Summary,Statistics,Residual,Length,,226.0
1,Summary,Statistics,Residual,# Missing Values,,0.0
2,Summary,Statistics,Residual,Mean,,-1028.1058
3,Summary,Statistics,Residual,Median,,-3048.68105
4,Summary,Statistics,Residual,Standard Deviation,,14688.910242
5,Summary,Statistics,Residual,Variance,,215764084.096414
6,Summary,Statistics,Residual,Kurtosis,,8.553715
7,Summary,Statistics,Residual,Skewness,,2.038166
8,Summary,Statistics,Residual,# Distinct Values,,226.0
9,White Noise,Ljung-Box,Residual,Test Statictic,"{'alpha': 0.05, 'K': 24}",26.165327


## ARIMA Model

In [17]:
arima = exp.create_model('arima')
exp.plot_model(estimator = arima, plot = 'forecast', 
               data_kwargs = {'fh' : 3})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2024-02-19/2024-02-25,0.3250,0.2687,4007.8011,5223.6030,0.3214,0.4160,-1.1461
1,2024-03-11/2024-03-17,0.6360,0.4174,7762.6922,8060.9778,0.4709,0.6415,-8.3594
2,2024-04-01/2024-04-07,0.7784,0.5774,9427.8085,11082.4876,0.2921,0.3562,-0.6416
Mean,NaT,0.5798,0.4211,7066.1006,8122.3561,0.3615,0.4712,-3.3824
SD,NaT,0.1893,0.1261,2266.8701,2392.2733,0.0783,0.1228,3.5253


In [18]:
exp.check_stats(arima, test = 'all')

,Test,Test Name,Data,Property,Setting,Value
0,Summary,Statistics,Residual,Length,,226.0
1,Summary,Statistics,Residual,# Missing Values,,0.0
2,Summary,Statistics,Residual,Mean,,59.992302
3,Summary,Statistics,Residual,Median,,-199.0258
4,Summary,Statistics,Residual,Standard Deviation,,19042.092943
5,Summary,Statistics,Residual,Variance,,362601303.63977
6,Summary,Statistics,Residual,Kurtosis,,7.000278
7,Summary,Statistics,Residual,Skewness,,0.049651
8,Summary,Statistics,Residual,# Distinct Values,,226.0
9,White Noise,Ljung-Box,Residual,Test Statictic,"{'alpha': 0.05, 'K': 24}",95.650654


## Taking Best Model and Forecasting

In [19]:
tuned_model = exp.tune_model(arima) # auto arima

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2024-02-19/2024-02-25,0.3752,0.3004,4627.5308,5840.7430,0.3821,0.5085,-1.6831
1,2024-03-11/2024-03-17,0.0947,0.0794,1156.3711,1532.7410,0.0780,0.0845,0.6616
2,2024-04-01/2024-04-07,0.7596,0.5558,9200.0123,10667.1604,0.2894,0.3474,-0.5208
Mean,NaT,0.4099,0.3118,4994.6381,6013.5481,0.2499,0.3135,-0.5141
SD,NaT,0.2725,0.1947,3294.0468,3731.1125,0.1273,0.1748,0.9573


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.8s finished


In [20]:
final_model = exp.finalize_model(tuned_model)
pred_holdout = exp.predict_model(final_model)

In [21]:
exp.plot_model(final_model)